In [ ]:
from pathlib import Path
import datetime
from zoneinfo import ZoneInfo
import shutil
from neuroconv.utils import load_dict_from_file, dict_deep_update
import h5py
import numpy as np
from pynwb import NWBHDF5IO, NWBFile

In [32]:
import pynwb

In [ ]:
from manimoh_nwb_converters import OdorSeqNWBConverter


In [8]:
file = h5py.File('E:\\odor-pixels\\M541-2024-08-31\\imec0_clean_lfp.mat', 'r') 

In [9]:
file['imec0'].keys()

<KeysViewHDF5 ['channel_ids', 'depths', 'lfp_fs', 'lfp_traces', 'lfp_tvec', 'shank_ids']>

In [10]:
# Code to get channel ids in array
q = np.asarray(file['imec0']['channel_ids'][:], dtype='uint32')
q = q.T.view('U1')
channel_ids = np.asarray([''.join(x).strip() for x in q])
channel_ids

array(['imec0.ap#AP112', 'imec0.ap#AP126', 'imec0.ap#AP138',
       'imec0.ap#AP200', 'imec0.ap#AP214', 'imec0.ap#AP226',
       'imec0.ap#AP288', 'imec0.ap#AP302', 'imec0.ap#AP162',
       'imec0.ap#AP176', 'imec0.ap#AP188', 'imec0.ap#AP250',
       'imec0.ap#AP264', 'imec0.ap#AP276', 'imec0.ap#AP338',
       'imec0.ap#AP352', 'imec0.ap#AP304', 'imec0.ap#AP318',
       'imec0.ap#AP330', 'imec0.ap#AP8', 'imec0.ap#AP22', 'imec0.ap#AP34',
       'imec0.ap#AP96', 'imec0.ap#AP110', 'imec0.ap#AP354',
       'imec0.ap#AP368', 'imec0.ap#AP380', 'imec0.ap#AP58',
       'imec0.ap#AP72', 'imec0.ap#AP84', 'imec0.ap#AP146',
       'imec0.ap#AP160'], dtype='<U14')

In [11]:
# Code to get shank_ids in array
shank_id = file['imec0']['shank_ids'][:].flatten()
shank_id


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3])

In [12]:
# Code to get depths ids in array TODO: Needs subtraction from ExpKeys.recordingDepth
depths = file['imec0']['depths'][:].flatten()
depths

array([ 480.,  585.,  675.,  780.,  885.,  975., 1080., 1185.,  495.,
        600.,  690.,  795.,  900.,  990., 1095., 1200.,  480.,  585.,
        675.,  780.,  885.,  975., 1080., 1185.,  495.,  600.,  690.,
        795.,  900.,  990., 1095., 1200.])

In [13]:
# Code to get lfp_fs
lfp_fs = file['imec0']['lfp_fs'][:].flatten()
lfp_fs[0]

2500.0

In [14]:
# Code to get lfp_tvec in array
lfp_tvec = file['imec0']['lfp_tvec'][:].flatten()
lfp_tvec

array([0.0000000e+00, 4.0000000e-04, 8.0000000e-04, ..., 6.1901092e+03,
       6.1901096e+03, 6.1901100e+03])

In [15]:
# Code to get lfp_traces in array
lfp_data= file['imec0']['lfp_traces'][:]
lfp_data = np.transpose(lfp_data)
lfp_data

array([[ -30.273438 ,  -48.4375   ,  -33.30078  , ...,  -18.164062 ,
          -6.0546875,  -27.246094 ],
       [  48.4375   ,  169.53125  ,    6.0546875, ...,   93.84766  ,
          84.765625 ,  -27.246094 ],
       [ 102.92969  ,  320.89844  ,   30.273438 , ...,  169.53125  ,
         151.36719  ,  -27.246094 ],
       ...,
       [-118.06641  , -136.23047  ,  -18.164062 , ..., -163.47656  ,
        -193.75     , -133.20312  ],
       [-105.95703  , -112.01172  ,    9.082031 , ..., -148.33984  ,
        -190.72266  , -124.12109  ],
       [ -93.84766  ,  -81.73828  ,   36.328125 , ..., -127.14844  ,
        -181.64062  , -112.01172  ]], dtype=float32)

In [21]:
from uuid import uuid4
from datetime import datetime
from dateutil.tz import tzlocal

In [ ]:
nwbfile = NWBFile(
    session_description="my first synthetic recording",
    identifier='-'.join([expkeys['subject'], expkeys['date']]),
    session_start_time=datetime.now(tzlocal()), # Get from .meta file
    experimenter=[
        ""Mohapatra, Manish","
    ],
    lab="vandermeerlab",
    institution="Dartmouth College",
    experiment_description="Head-fixed mouse presented with odor sequences",
    keywords=["ecephys", "exploration", "wanderlust"],
)
device = nwbfile.create_device(
    name="imec0", description="NPX2.0", manufacturer="IMEC"
)
nwbfile.add_electrode_column(name="channel_id", description="Identifier for the channel on the probe")

unique_shanks = np.unique(shank_id).tolist()
electrode_counter = 0
for iShank in unique_shanks:
    electrode_group = nwbfile.create_electrode_group(
        name="shank{}".format(iShank),
        description="electrode group for shank {}".format(iShank),
        device=device,
        location="brain area", # Need to figure this out, should this be the same as depth
    )
    # add electrodes to the electrode table
    for ielec,elec in enumerate(np.where(shank_id == iShank)[0]):
        # print
        nwbfile.add_electrode(
            group=electrode_group,
            channel_id = channel_ids[elec],
            location="brain area",  # Need to figure this out, should this be the same as depth
        )
        electrode_counter += 1

# nwbfile.electrodes.to_dataframe() # Distplay the electrode table

In [ ]:
lfp_table = nwbfile.create_electrode_table_region(
    region=list(range(electrode_counter)),  # reference row indices 0 to N-1
    description="LFP electrodes",
)

In [ ]:
from pynwb.ecephys import ElectricalSeries, LFP
lfp_es = ElectricalSeries(name='LFP', data=lfp_data, electrodes=lfp_table, rate=lfp_fs[0], starting_time=lfp_tvec[0])
ecephys_module = nwbfile.create_processing_module(
    name="ecephys", description="processed extracellular electrophysiology data"
)
ecephys_module.add(lfp_es)
# TODO: Add conversion factor from .meta file , but we already get 'scaled data' from spikeinterface'

In [ ]:
with NWBHDF5IO("E:\\odor-pixels\\M541-2024-08-31\\test.nwb", "w") as io:
    io.write(nwbfile)

In [35]:
with NWBHDF5IO("E:\\odor-pixels\\M541-2024-08-31\\test.nwb", "r") as io:
    nwbfile = io.read()

In [49]:
io = NWBHDF5IO("E:\\odor-pixels\\M541-2024-08-31\\test.nwb", "r")
nwb_file = io.read()
nwb_file

,location,group,group_name,channel_id
id,,,,
0,brain area,shank0 pynwb.ecephys.ElectrodeGroup at 0x2738738475840\nFields:\n description: electrode group for shank 0\n device: imec0 pynwb.device.Device at 0x2738738475168\nFields:\n description: NPX2.0\n manufacturer: IMEC\n\n location: brain area\n,shank0,imec0.ap#AP112
1,brain area,shank0 pynwb.ecephys.ElectrodeGroup at 0x2738738475840\nFields:\n description: electrode group for shank 0\n device: imec0 pynwb.device.Device at 0x2738738475168\nFields:\n description: NPX2.0\n manufacturer: IMEC\n\n location: brain area\n,shank0,imec0.ap#AP126
2,brain area,shank0 pynwb.ecephys.ElectrodeGroup at 0x2738738475840\nFields:\n description: electrode group for shank 0\n device: imec0 pynwb.device.Device at 0x2738738475168\nFields:\n description: NPX2.0\n manufacturer: IMEC\n\n location: brain area\n,shank0,imec0.ap#AP138
3,brain area,shank0 pynwb.ecephys.ElectrodeGroup at 0x2738738475840\nFields:\n description: electrode group for shank 0\n device: imec0 pynwb.device.Device at 0x2738738475168\nFields:\n description: NPX2.0\n manufacturer: IMEC\n\n location: brain area\n,shank0,imec0.ap#AP200
,location,group,group_name,channel_id
id,,,,
0,brain area,shank0 pynwb.ecephys.ElectrodeGroup at 0x2738738475840\nFields:\n description: electrode group for shank 0\n device: imec0 pynwb.device.Device at 0x2738738475168\nFields:\n description: NPX2.0\n manufacturer: IMEC\n\n location: brain area\n,shank0,imec0.ap#AP112
1,brain area,shank0 pynwb.ecephys.ElectrodeGroup at 0x2738738475840\nFields:\n description: electrode group for shank 0\n device: imec0 pynwb.device.Device at 0x2738738475168\nFields:\n description: NPX2.0\n manufacturer: IMEC\n\n location: brain area\n,shank0,imec0.ap#AP126
2,brain area,shank0 pynwb.ecephys.ElectrodeGroup at 0x2738738475840\nFields:\n description: electrode group for shank 0\n device: imec0 pynwb.device.Device at 0x2738738475168\nFields:\n description: NPX2.0\n manufacturer: IMEC\n\n location: brain area\n,shank0,imec0.ap#AP138


In [44]:
pynwb.validate(nwb_file)

TypeError: validate: incorrect type for 'io' (got 'NWBFile', expected 'HDMFIO')